In [4]:
# load scalp example
import trimesh
import numpy as np
import os
import json
import pyvista as pv
from electrodes_positions.utils.point_picking import pick_fiducials, pick_closed_path, project_fid_on_mesh

In [5]:
subjects_dir = '/srv/nfs-data/sisko/christian/SUBJECTS/'
input_mesh = 'AEGEUS_mathias/surfaces/charm_scalp.stl'
output_dir = 'AEGEUS_mathias/scalp_landmarks/'

In [6]:
input_mesh = subjects_dir + input_mesh
output_dir = subjects_dir + output_dir

mesh = trimesh.load(input_mesh)
vertices = np.array(mesh.vertices)
faces = np.array(mesh.faces)

# Picking points utils
## pick fiducials
With this utility you can choose the four fiducial landmarks on the subject's head

In [7]:
# pick fiducials using a GUI
picked_points = pick_fiducials(vertices, faces)

2026-01-27 15:45:11.013 (  80.004s) [    7C2C8F1A8740]vtkXOpenGLRenderWindow.:1416  WARN| bad X server connection. DISPLAY=localhost:11.0


Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7c2b1e0924b0_0" src="http://localhost:8890/trame…

In [8]:
# project the fiducials on the mesh vertices
(RPA, LPA, NAS, IN), (RPA_idx, LPA_idx, NAS_idx, IN_idx) = project_fid_on_mesh(picked_points, vertices, return_positions = True, return_indices=True)

## selects the three regions on which electrodes should not be placed
with this utils you can select closed paths on the head from which to remove missplaced electrodes (i.e. the ears and the eyes)

In [9]:
picked_points, paths = pick_closed_path(vertices, faces)

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7c2b01e3ad50_0" src="http://localhost:8890/trame…

In [10]:
right_ear = picked_points
picked_points, paths = pick_closed_path(vertices, faces)

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7c2b0ff98140_0" src="http://localhost:8890/trame…

In [11]:
left_ear = picked_points
picked_points, paths = pick_closed_path(vertices, faces)

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7c2b041aeb10_0" src="http://localhost:8890/trame…

In [12]:
eyes = picked_points

In [13]:
right_ear = vertices[right_ear]
left_ear = vertices[left_ear]
eyes = vertices[eyes]

# Save on disk
Ideally, these should go in \\$SUBJECTS_DIR/\\$SUBJECT/scalp_landmarks/

In [15]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [16]:
fiducials = {}
fiducials['RPA'] = list(RPA)
fiducials['LPA'] = list(LPA)
fiducials['NAS'] = list(NAS)
fiducials['IN'] = list(IN)

with open(os.path.join(output_dir, 'fiducials.json'), 'w') as f:
    json.dump(fiducials, f)

In [17]:
outlines = np.concatenate([right_ear, left_ear, eyes])

np.save(os.path.join(output_dir, 'outlines.npy'), outlines)